In [20]:
from pathlib import Path
from datetime import datetime, timedelta

def average_latency(sub_log: Path) -> float:
	readings = []
	with sub_log.open() as f:
		for line in f:
			if " - " in line:
				ts, val = line.split(" - ")
				readings.append((datetime.fromisoformat(ts), datetime.fromisoformat(val.strip())))

	# remove last value (is unusally large latency due to timeout)
	readings = readings[:-1]

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	# add 1s to values (cache flushing issue)
	readings = [(ts, val + timedelta(seconds=1))  for ts, val, in readings]


	latency = [(ts-val).total_seconds() for ts, val in readings]

	return sum(latency)/len(latency)

def average_cpu_usage(relay_perf: Path) -> float:
	readings = []
	with relay_perf.open() as f:
		for line in f:
			ts, val = line.split("   ")
			if val == "":
				raise ValueError("empty val")
			readings.append((datetime.fromisoformat(ts.strip()[:-1]), float(val.strip())))

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	percentages = [v for _, v in readings]

	return sum(percentages)/len(percentages)


def load_measurements(dir: Path):
	measurements = dir / Path("measurements")
	if not measurements.is_dir():
		raise ValueError(f"invalid directory: {measurements}")

	sub_logs = []
	relay_perf = []

	# organize files
	for file in measurements.iterdir():
		if file.name.startswith("clock"):
			sub_logs.append(file)
		if file.name.endswith("cpu_usage.log"):
			relay_perf.append(file)

	sub_avg_latencies = []
	for sub in sub_logs:
		try:
			sub_avg_latencies.append(average_latency(sub))
		except Exception as e:
			pass#print(f"{sub} skipped due to {e}")

	relay_perf_avg = {}
	for relay in relay_perf:
		name = relay.stem.replace("_cpu_usage", "")
		try:
			relay_perf_avg[name] = average_cpu_usage(relay)
		except Exception as e:
			print(f"{name} skipped due to {e}")


	return sub_avg_latencies, relay_perf_avg

load_measurements(Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement/star"))

([0.016178249999999995,
  0.016326960784313723,
  0.016621098039215687,
  0.016335596153846152,
  0.013658745098039214,
  0.013065673076923075,
  0.01223569230769231,
  0.012558980769230762,
  0.012379807692307692,
  0.012568980769230768,
  0.016411538461538464,
  0.017032480769230765,
  0.013225,
  0.009190403846153845,
  0.016243333333333332,
  0.008792235294117648,
  0.009105115384615385,
  0.008636470588235294,
  0.012908576923076922,
  0.015675057692307693,
  0.01236846153846154,
  0.01619239215686275,
  0.016427192307692302,
  0.01650403846153846,
  0.016033039215686277,
  0.016572557692307692,
  0.013764392156862745,
  0.015392673076923083,
  0.015433346153846148,
  0.012268,
  0.008527038461538464,
  0.016765803921568623,
  0.009685156862745098,
  0.013503588235294118,
  0.01630265384615385,
  0.015351843137254906,
  0.016436711538461542,
  0.008542692307692305,
  0.015614884615384612,
  0.01632346153846154,
  0.016382173076923072,
  0.012829196078431374,
  0.01639830769230769,

In [22]:
from pprint import pprint
base = Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement")
for topo in ["full-mesh", "star", "leaf-and-spine", "two-relays", "single-relay"]:
	latencies, relays = load_measurements(base / topo)

	avg_lat = sum(latencies) / len(latencies)
	pprint({
		"name": topo,
		"average_latency": avg_lat,
		"relay_utilizations": relays
	})

{'average_latency': 0.015293579401306236,
 'name': 'full-mesh',
 'relay_utilizations': {'relay1': 0.5541666666666667,
                        'relay2': 0.5979166666666667,
                        'relay3': 0.5583333333333333,
                        'relay4': 0.55625,
                        'relay5': 0.5854166666666668}}
{'average_latency': 0.013716340502262458,
 'name': 'star',
 'relay_utilizations': {'hub': 0.42244897959183686,
                        'relay1': 0.33469387755102037,
                        'relay2': 0.3081632653061224,
                        'relay3': 0.3061224489795918,
                        'relay4': 0.30816326530612237}}
{'average_latency': 0.010897928085218703,
 'name': 'leaf-and-spine',
 'relay_utilizations': {'leaf1': 0.48999999999999977,
                        'leaf2': 0.48200000000000004,
                        'leaf3': 0.45599999999999985,
                        'spine1': 0.48200000000000004,
                        'spine2': 0.5079999999999999}}
{'ave